In [1]:
# Cloud usage for data storage in EU countries 2017-2018
# analysis in DataBricks (Pyton kernel)

In [2]:
%scala
import org.apache.spark.sql._
val cloudDF = spark.read.
 format("org.apache.spark.csv")
 .option("header", true)
 .option("inferSchema", true)
 .option("sep", ",")
 .csv("/FileStore/tables/cloud_usage.csv").cache()
// cloudDF = cloudDF.dropna() # drop rows with missing values
cloudDF.createOrReplaceTempView("cloud")
cloudDF.printSchema

root
-- TIME: integer (nullable = true)
-- GEO: string (nullable = true)
-- INDIC_IS: string (nullable = true)
-- UNIT: string (nullable = true)
-- IND_TYPE: string (nullable = true)
-- Value: integer (nullable = true)

import org.apache.spark.sql._
cloudDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [TIME: int, GEO: string ... 4 more fields]

In [3]:
%sql
SELECT count(distinct time) as years
FROM cloud
-- 3 years included

years
3


In [4]:
%sql
SELECT time as years, count(*) as countries
FROM cloud
GROUP BY time
ORDER BY time DESC
-- 3 years listed, for 29 countries

years,countries
2019,29
2018,29
2017,29


In [5]:
%sql
SELECT * FROM (
  SELECT geo, time, value, 
    DENSE_RANK() OVER ( 
    ORDER BY value DESC) as dense_rank 
  FROM cloud) outertable
-- WHERE time = 2019;

GEO,TIME,Value,dense_rank
Sweden,2018,64,1
Denmark,2019,64,1
Sweden,2019,64,1
Denmark,2018,62,2
Sweden,2017,59,3
Denmark,2017,56,4
Ireland,2019,55,5
Netherlands,2019,54,6
United Kingdom,2019,53,7
Ireland,2017,51,8


In [6]:
%sql
SELECT * FROM (
  SELECT geo, time, value, 
  RANK() OVER ( 
  ORDER BY value DESC) ranking FROM cloud) outertable

GEO,TIME,Value,ranking
Sweden,2018,64,1
Denmark,2019,64,1
Sweden,2019,64,1
Denmark,2018,62,4
Sweden,2017,59,5
Denmark,2017,56,6
Ireland,2019,55,7
Netherlands,2019,54,8
United Kingdom,2019,53,9
Ireland,2017,51,10


In [7]:
%sql
SELECT * FROM (
  SELECT geo, time, value, 
  RANK() OVER ( 
  ORDER BY value DESC) ranking FROM cloud) AS outertable
WHERE TIME = "2019"

GEO,TIME,Value,ranking
Denmark,2019,64,1
Sweden,2019,64,1
Ireland,2019,55,7
Netherlands,2019,54,8
United Kingdom,2019,53,9
Luxembourg,2019,49,14
Cyprus,2019,48,19
Malta,2019,47,20
Estonia,2019,46,21
Spain,2019,46,21


In [8]:
%sql
SELECT geo, time, value, 
  RANK() OVER ( 
    PARTITION BY time
    ORDER BY value DESC)
    AS ranking 
FROM cloud
GROUP BY geo, time, value

geo,time,value,ranking
Sweden,2018,64,1
Denmark,2018,62,2
Netherlands,2018,51,3
United Kingdom,2018,50,4
Finland,2018,49,5
Ireland,2018,49,5
Luxembourg,2018,46,7
Malta,2018,46,7
Hungary,2018,43,9
Spain,2018,40,10


In [9]:
%sql
SELECT * FROM (
  SELECT geo, time, value, 
    CUME_DIST() OVER ( 
    ORDER BY value DESC) cume_dist 
  FROM cloud) AS outertable
WHERE TIME = "2019"
-- calculates the relative position of a value in a group of values

GEO,TIME,Value,ranking
Denmark,2019,64,0.034482758620689655
Sweden,2019,64,0.034482758620689655
Ireland,2019,55,0.08045977011494253
Netherlands,2019,54,0.09195402298850575
United Kingdom,2019,53,0.10344827586206896
Luxembourg,2019,49,0.20689655172413793
Cyprus,2019,48,0.21839080459770116
Malta,2019,47,0.22988505747126436
Estonia,2019,46,0.28735632183908044
Spain,2019,46,0.28735632183908044


In [10]:
%sql
SELECT geo, time, value, 
  100*CUME_DIST() OVER ( 
    ORDER BY value DESC)
    AS cume_dist 
FROM cloud
WHERE time = '2019'
GROUP BY geo, time, value
LIMIT 8

geo,time,value,cume_dist
Denmark,2019,64,6.896551724137931
Sweden,2019,64,6.896551724137931
Ireland,2019,55,10.344827586206897
Netherlands,2019,54,13.793103448275861
United Kingdom,2019,53,17.24137931034483
Luxembourg,2019,49,20.689655172413794
Cyprus,2019,48,24.137931034482758
Malta,2019,47,27.586206896551722


In [11]:
%sql
SELECT geo, time, value, 
    ROUND(CUME_DIST() OVER ( 
    ORDER BY value DESC),4)
    AS cume_dist,
    ROUND(100*CUME_DIST() OVER ( 
    ORDER BY value DESC),2)
    AS proc
FROM cloud
WHERE time = '2019'
GROUP BY geo, time, value

-- CUME_DIST() shows how much % is in a partition 
-- precedes current record in a ranking (percentyl)
-- no need for PARTITION BY

geo,time,value,cume_dist,proc
Denmark,2019,64,0.069,6.9
Sweden,2019,64,0.069,6.9
Ireland,2019,55,0.1034,10.34
Netherlands,2019,54,0.1379,13.79
United Kingdom,2019,53,0.1724,17.24
Luxembourg,2019,49,0.2069,20.69
Cyprus,2019,48,0.2414,24.14
Malta,2019,47,0.2759,27.59
Finland,2019,46,0.3793,37.93
Spain,2019,46,0.3793,37.93


In [12]:
%sql
WITH top20 AS (
  SELECT geo, time, value, 
      ROUND(100*CUME_DIST() OVER ( 
      PARTITION BY time
      ORDER BY value DESC),2)
      AS cume_dist
  FROM cloud
  )
SELECT * FROM top20
WHERE cume_dist <= 20;
  
-- gets top 20% of all cloud users

geo,time,value,cume_dist
Sweden,2018,64,3.45
Denmark,2018,62,6.9
Netherlands,2018,51,10.34
United Kingdom,2018,50,13.79
Denmark,2019,64,6.9
Sweden,2019,64,6.9
Ireland,2019,55,10.34
Netherlands,2019,54,13.79
United Kingdom,2019,53,17.24
Sweden,2017,59,3.45


In [13]:
%sql
WITH top20 AS (
  SELECT geo, time, value, 
      ROUND(100*CUME_DIST() OVER ( 
      ORDER BY value DESC),2)
      AS cume_dist
  FROM cloud
  WHERE time = '2019'
  )
SELECT * FROM top20
WHERE 
    cume_dist <= 20;
  
-- gets top 20% of all cloud users in 2019

geo,time,value,cume_dist
Denmark,2019,64,6.9
Sweden,2019,64,6.9
Ireland,2019,55,10.34
Netherlands,2019,54,13.79
United Kingdom,2019,53,17.24


In [14]:
%sql
SELECT * FROM (
  SELECT GEO, TIME, Value, 
    ROUND(CUME_DIST() OVER ( 
    PARTITION BY time
    ORDER BY Value DESC),3) cume_dist 
  FROM cloud) AS outertable

GEO,TIME,Value,cume_dist
Sweden,2018,64,0.034
Denmark,2018,62,0.069
Netherlands,2018,51,0.103
United Kingdom,2018,50,0.138
Finland,2018,49,0.207
Ireland,2018,49,0.207
Luxembourg,2018,46,0.276
Malta,2018,46,0.276
Hungary,2018,43,0.31
Estonia,2018,40,0.379


In [15]:
%sql
SELECT geo, time, value, 
    ROUND(CUME_DIST() OVER ( 
    ORDER BY value DESC),4)
    AS cume_dist,
    ROUND(100*CUME_DIST() OVER ( 
    ORDER BY value DESC),2)
    AS cd_proc,
    ROUND(PERCENT_RANK() OVER ( 
    ORDER BY value DESC),2)
    AS perc_rank
FROM cloud
WHERE time = '2019'
GROUP BY geo, time, value
-- PERC_RANK skips current reckord (0)

geo,time,value,cume_dist,cd_proc,perc_rank
Denmark,2019,64,0.069,6.9,0.0
Sweden,2019,64,0.069,6.9,0.0
Ireland,2019,55,0.1034,10.34,0.07
Netherlands,2019,54,0.1379,13.79,0.11
United Kingdom,2019,53,0.1724,17.24,0.14
Luxembourg,2019,49,0.2069,20.69,0.18
Cyprus,2019,48,0.2414,24.14,0.21
Malta,2019,47,0.2759,27.59,0.25
Finland,2019,46,0.3793,37.93,0.29
Spain,2019,46,0.3793,37.93,0.29


In [16]:
%sql
SELECT geo, time, value, 
    NTILE(4) OVER ( 
    ORDER BY value DESC)
    AS ntile 
FROM cloud
WHERE time = '2019'
-- no partitioning

geo,time,value,ntile
Denmark,2019,64,1
Sweden,2019,64,1
Ireland,2019,55,1
Netherlands,2019,54,1
United Kingdom,2019,53,1
Luxembourg,2019,49,1
Cyprus,2019,48,1
Malta,2019,47,1
Estonia,2019,46,2
Spain,2019,46,2
